# Common imports

In [253]:
%matplotlib inline
import os
import sys
import glob
import re
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import Image
import timeit

# Provided functions
### Image loading and saving

In [254]:
def LoadFaceImages(pathname, subject_name, num_images):
    """
    Load the set of face images.  
    The routine returns
        ambimage: image illuminated under the ambient lighting
        imarray: a 3-D array of images, h x w x Nimages
        lightdirs: Nimages x 3 array of light source directions
    """

    def load_image(fname):
        return np.asarray(Image.open(fname))

    def fname_to_ang(fname):
        yale_name = os.path.basename(fname)
        return int(yale_name[12:16]), int(yale_name[17:20])

    def sph2cart(az, el, r):
        rcos_theta = r * np.cos(el)
        x = rcos_theta * np.cos(az)
        y = rcos_theta * np.sin(az)
        z = r * np.sin(el)
        return x, y, z

    ambimage = load_image(
        os.path.join(pathname, subject_name + '_P00_Ambient.pgm'))
    im_list = glob.glob(os.path.join(pathname, subject_name + '_P00A*.pgm'))
    if num_images <= len(im_list):
        im_sub_list = np.random.choice(im_list, num_images, replace=False)
    else:
        print(
            'Total available images is less than specified.\nProceeding with %d images.\n'
            % len(im_list))
        im_sub_list = im_list
    im_sub_list.sort()
    imarray = np.stack([load_image(fname) for fname in im_sub_list], axis=-1)
    Ang = np.array([fname_to_ang(fname) for fname in im_sub_list])

    x, y, z = sph2cart(Ang[:, 0] / 180.0 * np.pi, Ang[:, 1] / 180.0 * np.pi, 1)
    lightdirs = np.stack([y, z, x], axis=-1)
    return ambimage, imarray, lightdirs

In [255]:
def save_outputs(subject_name, albedo_image, surface_normals):
    im = Image.fromarray((albedo_image*255).astype(np.uint8))
    im.save("%s_albedo.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,0]*128+128).astype(np.uint8))
    im.save("%s_normals_x.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,1]*128+128).astype(np.uint8))
    im.save("%s_normals_y.jpg" % subject_name)
    im = Image.fromarray((surface_normals[:,:,2]*128+128).astype(np.uint8))
    im.save("%s_normals_z.jpg" % subject_name)

### Plot the height map

In [256]:
def set_aspect_equal_3d(ax):
    """https://stackoverflow.com/questions/13685386"""
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    from numpy import mean
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([
        abs(lim - mean_)
        for lims, mean_ in ((xlim, xmean), (ylim, ymean), (zlim, zmean))
        for lim in lims
    ])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])


def display_output(albedo_image, height_map):
    fig = plt.figure()
    plt.imshow(albedo_image, cmap='gray')
    plt.axis('off')
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 50)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

In [257]:
def display_output_multiview(height_map):  
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(20, 50)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(20, -50)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(-5, 50)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)
    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca(projection='3d')
    ax.view_init(-5, -50)
    X = np.arange(albedo_image.shape[0])
    Y = np.arange(albedo_image.shape[1])
    X, Y = np.meshgrid(Y, X)
    H = np.flipud(np.fliplr(height_map))
    A = np.flipud(np.fliplr(albedo_image))
    A = np.stack([A, A, A], axis=-1)
    ax.xaxis.set_ticks([])
    ax.xaxis.set_label_text('Z')
    ax.yaxis.set_ticks([])
    ax.yaxis.set_label_text('X')
    ax.zaxis.set_ticks([])
    ax.yaxis.set_label_text('Y')
    surf = ax.plot_surface(
        H, X, Y, cmap='gray', facecolors=A, linewidth=0, antialiased=False)
    set_aspect_equal_3d(ax)

### Plot the surface norms. 

In [258]:
def plot_surface_normals(surface_normals):
    """
    surface_normals: h x w x 3 matrix.
    """
    fig = plt.figure()
    ax = plt.subplot(1, 3, 1)
    ax.axis('off')
    ax.set_title('X')
    im = ax.imshow(surface_normals[:,:,0],cmap='jet',vmin=-1.0, vmax=1.0)
    
    ax = plt.subplot(1, 3, 2)
    ax.axis('off')
    ax.set_title('Y')
    im = ax.imshow(surface_normals[:,:,1],cmap='jet',vmin=-1.0, vmax=1.0)
    ax = plt.subplot(1, 3, 3)
    ax.axis('off')
    ax.set_title('Z')
    im = ax.imshow(surface_normals[:,:,2],cmap='jet',vmin=-1.0, vmax=1.0)

# Your implementation

In [259]:
def preprocess(ambimage, imarray): 
    """
    preprocess the data: 
        1. subtract ambient_image from each image in imarray.
        2. make sure no pixel is less than zero.
        3. rescale values in imarray to be between 0 and 1.
    Inputs:
        ambimage: h x w
        imarray: h x w x Nimages
    Outputs:
        processed_imarray: h x w x Nimages
    """
    processed_imarray = imarray-ambimage[:,:,None]
    processed_imarray[processed_imarray<0] = 0
    processed_imarray= processed_imarray/np.max(processed_imarray)

    return processed_imarray

In [260]:
def photometric_stereo(imarray, light_dirs):
    """
    Inputs:
        imarray:  h x w x Nimages
        light_dirs: Nimages x 3
    Outputs:
        albedo_image: h x w
        surface_norms: h x w x 3
    """
# Solving multiple linear system appraoch (uncommet to execute)
#     albedo_image = np.zeros([imarray.shape[0],imarray.shape[1]])
#     surface_normals = np.zeros([imarray.shape[0],imarray.shape[1],3])
#     for i in range(imarray.shape[0]):
#         for j in range(imarray.shape[1]):
#             b = imarray[i,j,:]
#             A = light_dirs
#             g,_,_,_ = np.linalg.lstsq(A,b,rcond = None)
#             rho = np.sqrt(g[0]**2+g[1]**2+g[2]**2)
#             albedo_image[i,j] = rho
#             surface_normals[i,j,0] = g[0]/rho
#             surface_normals[i,j,1] = g[1]/rho
#             surface_normals[i,j,2] = g[2]/rho

# One linear system by stacking vectors 
    height, width, Nimages = imarray.shape
    npix = height*width
    I = np.zeros([Nimages,npix])
    for i in range(Nimages):
        I[i,:] = np.reshape(imarray[:,:,i],[1,npix])
    
    g,_,_,_ = np.linalg.lstsq(light_dirs,I,rcond = None)
    albedo = np.zeros([1,npix])
    normal = np.zeros([3,npix])
    for i in range(npix):
        albedo[:,i] = np.linalg.norm(g[:,i])
        normal[:,i] = g[:,i]/albedo[:,i]
    
    albedo_image = np.reshape(albedo,[height,width])
    surface_normals = np.zeros([imarray.shape[0],imarray.shape[1],3])
    surface_normals[:,:,0] = np.reshape(normal[0,:],[height,width])
    surface_normals[:,:,1] = np.reshape(normal[1,:],[height,width])
    surface_normals[:,:,2] = np.reshape(normal[2,:],[height,width])



            
    return albedo_image, surface_normals

In [261]:
def get_surface(surface_normals, integration_method):
    """
    Inputs:
        surface_normals:h x w x 3
        integration_method: string in ['average', 'column', 'row', 'random']
    Outputs:
        height_map: h x w
    """ 
    height_map = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
    
    fx = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
    fy = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
    
    fx = surface_normals[:,:,0]/surface_normals[:,:,2]
    fy = surface_normals[:,:,1]/surface_normals[:,:,2]
    
    if (integration_method=='row'):
        temp = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
        temp[1:-1,0] = np.cumsum(fy[1:-1,0])
        temp[:,1:-1] = fx[:,1:-1]
        height_map = np.cumsum(temp,axis=1)
    elif(integration_method=='column'):
        temp = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
        temp[0,1:-1] = np.cumsum(fx[0,1:-1])
        temp[1:-1,:] = fy[1:-1,:]
        height_map = np.cumsum(temp,axis=0)
    elif(integration_method=='average'):
        temp = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
        temp[1:-1,0] = np.cumsum(fy[1:-1,0])
        temp[:,1:-1] = fx[:,1:-1]
        height_map_1 = np.cumsum(temp,axis=1)
        
        temp = np.zeros([surface_normals.shape[0],surface_normals.shape[1]])
        temp[0,1:-1] = np.cumsum(fx[0,1:-1])
        temp[1:-1,:] = fy[1:-1,:]
        height_map_2 = np.cumsum(temp,axis=0)
        
        height_map = 0.5*(height_map_1+height_map_2)
    elif(integration_method=='random'):
        # of random paths
        n=20
        height = surface_normals.shape[0]
        width = surface_normals.shape[1]
        # Initialize height_map
        height_map = np.zeros([height, width])
        temp = height_map
        for row in range(height):
            for col in range(width):
                if (row == 0 and col == 0):
                    height_map[row, col] = 0;
                else:
                    for i in range(n):
                        csum = 0
                        count = 0;
                        R = np.random.randint(1,3,size = row+col+1)
                        w = 0;
                        h = 0;
                        while(w < col or h < row):
                            if (w >= col):
                                R[count]=2
                            if (h >= row):
                                R[count] = 1;
                            if (R[count] == 1):
                                csum = csum + fx[h,w]
                                w = w + 1
                            else:
                                csum = csum + fy[h,w]
                                h = h + 1
                            count = count + 1
                        height_map[row, col] = csum + height_map[row, col] 
                    height_map[row, col] = height_map[row, col] / n;
  
    return height_map

# Main function

In [ ]:
# use subset of images. 

root_path = './croppedyale/'
subject_name = 'yaleB02'
#integration_method = 'average'
save_flag = True
full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,64)
processed_imarray = preprocess(ambient_image, imarray)
albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)
height_map = get_surface(surface_normals, 'random')
plot_surface_normals(surface_normals)
display_output(albedo_image, height_map)


if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

In [262]:
## Calcualte the performace for each integratio method 

root_path = './croppedyale/'
subject_name = 'yaleB02'
#integration_method = 'average'
save_flag = False
full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,64)
processed_imarray = preprocess(ambient_image, imarray)
albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)
time_elapse = []
n = 0
while (n<=10):
    tic = timeit.default_timer();
    height_map = get_surface(surface_normals, 'average')
    toc = timeit.default_timer();
    time_elapse.append(toc-tic)
    n = n+1
print("time elapse = ", sum(time_elapse)/len(time_elapse))

time elapse =  0.0004836930999193679


In [263]:
# use subset of images. 

root_path = './croppedyale_subset/'
subject_name = 'yaleB02'
#integration_method = 'average'
save_flag = True
full_path = '%s%s' % (root_path, subject_name)
ambient_image, imarray, light_dirs = LoadFaceImages(full_path, subject_name,64)
processed_imarray = preprocess(ambient_image, imarray)
albedo_image, surface_normals = photometric_stereo(processed_imarray, light_dirs)
height_map = get_surface(surface_normals, 'random')
plot_surface_normals(surface_normals)
display_output(albedo_image, height_map)


if save_flag:
    save_outputs(subject_name, albedo_image, surface_normals)

Total available images is less than specified.
Proceeding with 54 images.

